Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  regularization = 5e-4 * tf.nn.l2_loss(weights)
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [ ]:
batch_size = 128
relu_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_size]))
  biases_in = tf.Variable(tf.zeros([relu_size]))
    
  weights_out = tf.Variable(
    tf.truncated_normal([relu_size, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_in = tf.matmul(tf_train_dataset, weights_in) + biases_in
  logits_out = tf.nn.relu(logits_in)
  logits = tf.matmul(logits_out, weights_out) + biases_out

  regularization = 5e-4 * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out)) 
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
    
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #Valid
  logits_in = tf.matmul(tf_valid_dataset, weights_in) + biases_in
  logits_out = tf.nn.relu(logits_in)
  logits = tf.matmul(logits_out, weights_out) + biases_out
  valid_prediction = tf.nn.softmax(logits)
  #Test
  logits_in = tf.matmul(tf_test_dataset, weights_in) + biases_in
  logits_out = tf.nn.relu(logits_in)
  logits = tf.matmul(logits_out, weights_out) + biases_out
  test_prediction = tf.nn.softmax(logits)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
batch_size = 10
relu_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_size]))
  biases_in = tf.Variable(tf.zeros([relu_size]))
    
  weights_out = tf.Variable(
    tf.truncated_normal([relu_size, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_in = tf.matmul(tf_train_dataset, weights_in) + biases_in
  logits_out = tf.nn.relu(logits_in)
  logits = tf.matmul(logits_out, weights_out) + biases_out

  regularization = 0#5e-4 * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out)) 
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization
    
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #Valid
  logits_in = tf.matmul(tf_valid_dataset, weights_in) + biases_in
  logits_out = tf.nn.relu(logits_in)
  logits = tf.matmul(logits_out, weights_out) + biases_out
  valid_prediction = tf.nn.softmax(logits)
  #Test
  logits_in = tf.matmul(tf_test_dataset, weights_in) + biases_in
  logits_out = tf.nn.relu(logits_in)
  logits = tf.matmul(logits_out, weights_out) + biases_out
  test_prediction = tf.nn.softmax(logits)

In [12]:
num_steps = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 424.647400
Minibatch accuracy: 10.0%
Validation accuracy: 13.6%
Minibatch loss at step 10: 4089.841064
Minibatch accuracy: 40.0%
Validation accuracy: 33.1%
Minibatch loss at step 20: 2071.373779
Minibatch accuracy: 50.0%
Validation accuracy: 54.4%
Minibatch loss at step 30: 1528.143188
Minibatch accuracy: 50.0%
Validation accuracy: 45.0%
Minibatch loss at step 40: 1765.935913
Minibatch accuracy: 50.0%
Validation accuracy: 53.6%
Minibatch loss at step 50: 1155.399048
Minibatch accuracy: 40.0%
Validation accuracy: 41.8%
Minibatch loss at step 60: 724.065063
Minibatch accuracy: 60.0%
Validation accuracy: 52.2%
Minibatch loss at step 70: 3179.750488
Minibatch accuracy: 50.0%
Validation accuracy: 47.8%
Minibatch loss at step 80: 11473.712891
Minibatch accuracy: 40.0%
Validation accuracy: 43.5%
Minibatch loss at step 90: 4405.915527
Minibatch accuracy: 60.0%
Validation accuracy: 46.9%
Test accuracy: 49.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [34]:
batch_size = 128
relu_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_size]))
  biases_in = tf.Variable(tf.zeros([relu_size]))
    
  weights_out = tf.Variable(
    tf.truncated_normal([relu_size, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computationы
  logits_in = tf.matmul(tf_train_dataset, weights_in) + biases_in
  drop_in = tf.nn.relu(logits_in)
  logits_out = tf.nn.dropout(drop_in, tf.constant(0.5))
  logits = tf.matmul(logits_out, weights_out) + biases_out
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #Valid
  logits_in = tf.matmul(tf_valid_dataset, weights_in) + biases_in
  drop_in = tf.nn.relu(logits_in)
  logits_out = tf.nn.dropout(drop_in, tf.constant(1.))
  logits = tf.matmul(logits_out, weights_out) + biases_out
  valid_prediction = tf.nn.softmax(logits)
  #Test
  logits_in = tf.matmul(tf_test_dataset, weights_in) + biases_in
  drop_in = tf.nn.relu(logits_in)
  logits_out = tf.nn.dropout(drop_in, tf.constant(1.))
  logits = tf.matmul(logits_out, weights_out) + biases_out
  test_prediction = tf.nn.softmax(logits)

In [39]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 443.853638
Minibatch accuracy: 14.1%
Validation accuracy: 26.9%
Minibatch loss at step 500: 21.006687
Minibatch accuracy: 79.7%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 16.589804
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 10.795641
Minibatch accuracy: 73.4%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 12.428283
Minibatch accuracy: 76.6%
Validation accuracy: 79.1%
Minibatch loss at step 2500: 5.912678
Minibatch accuracy: 71.1%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 11.159372
Minibatch accuracy: 67.2%
Validation accuracy: 78.4%
Minibatch loss at step 3500: 13.496180
Minibatch accuracy: 70.3%
Validation accuracy: 79.6%
Minibatch loss at step 4000: 1.622892
Minibatch accuracy: 77.3%
Validation accuracy: 80.0%
Minibatch loss at step 4500: 3.971728
Minibatch accuracy: 75.8%
Validation accuracy: 80.1%
Minibatch loss at step 5000: 2.569271
Minibatch accuracy: 81.2%
Validation

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [43]:
batch_size = 128
relu_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_size]))
  biases_in = tf.Variable(tf.zeros([relu_size]))
    
  weights_out = tf.Variable(
    tf.truncated_normal([relu_size, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computationы
  logits_in = tf.matmul(tf_train_dataset, weights_in) + biases_in
  drop_in = tf.nn.relu(logits_in)
  logits_out = tf.nn.dropout(drop_in, tf.constant(0.5))
  logits = tf.matmul(logits_out, weights_out) + biases_out
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  #Valid
  logits_in = tf.matmul(tf_valid_dataset, weights_in) + biases_in
  drop_in = tf.nn.relu(logits_in)
  logits_out = tf.nn.dropout(drop_in, tf.constant(1.))
  logits = tf.matmul(logits_out, weights_out) + biases_out
  valid_prediction = tf.nn.softmax(logits)
  #Test
  logits_in = tf.matmul(tf_test_dataset, weights_in) + biases_in
  drop_in = tf.nn.relu(logits_in)
  logits_out = tf.nn.dropout(drop_in, tf.constant(1.))
  logits = tf.matmul(logits_out, weights_out) + biases_out
  test_prediction = tf.nn.softmax(logits)

In [45]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    #global_step = tf.Variable(step)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 478.905457
Minibatch accuracy: 7.8%
Validation accuracy: 27.1%
Minibatch loss at step 500: 11.140136
Minibatch accuracy: 76.6%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 11.785902
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 8.640585
Minibatch accuracy: 68.8%
Validation accuracy: 80.1%
Minibatch loss at step 2000: 16.690601
Minibatch accuracy: 71.1%
Validation accuracy: 79.4%
Minibatch loss at step 2500: 6.837223
Minibatch accuracy: 78.1%
Validation accuracy: 79.4%
Minibatch loss at step 3000: 9.339245
Minibatch accuracy: 71.9%
Validation accuracy: 79.8%
Minibatch loss at step 3500: 13.291487
Minibatch accuracy: 66.4%
Validation accuracy: 80.2%
Minibatch loss at step 4000: 2.362170
Minibatch accuracy: 77.3%
Validation accuracy: 79.9%
Minibatch loss at step 4500: 4.121241
Minibatch accuracy: 75.0%
Validation accuracy: 80.0%
Minibatch loss at step 5000: 3.550573
Minibatch accuracy: 78.9%
Validation ac